In [ ]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
import time
import warnings
from datetime import datetime

# Suppress warnings
warnings.filterwarnings("ignore")

# Define the file paths
historical_file_2023 = r'E:\Data_Practice\xrp_rate_custom_2023-01-01_to_2024-01-01.csv'
historical_file_2024 = r'E:\Data_Practice\xrp_rate_custom_2024-01-01_to_2024-10-21.csv'
current_day_file = r'E:\Data_Practice\xrp_rates_live.csv'
model_weights_file = r'E:\Data_Practice\lstm_checkpoint_weights.weights.h5'
model_file = r'E:\Data_Practice\lstm_model.h5'
forecast_file = r'E:\Data_Practice\xrp_forecast.csv'

# Load and combine historical data
def load_historical_data():
    """
    Load and combine historical data from 2023 and 2024.
    Raises a FileNotFoundError if any of the files are missing.
    """
    if os.path.exists(historical_file_2023) and os.path.exists(historical_file_2024):
        historical_data_2023 = pd.read_csv(historical_file_2023, parse_dates=['Timestamp'])
        historical_data_2024 = pd.read_csv(historical_file_2024, parse_dates=['Timestamp'])
        combined_data = pd.concat([historical_data_2023, historical_data_2024], ignore_index=True)
        print("Loaded historical data successfully.")
        return combined_data
    else:
        raise FileNotFoundError("One of the historical data files is missing.")

# Preprocess data for training
def preprocess_for_training(data):
    """
    Preprocess the data for training.
    Returns the features (X) and target (y) arrays for the LSTM model.
    """
    data['XRP Price'] = data['XRP Price'].astype(float)
    X, y = [], []
    for i in range(len(data) - 1):
        X.append(data['XRP Price'].iloc[i])
        y.append(data['XRP Price'].iloc[i + 1])
    
    X = np.array(X).reshape((-1, 1, 1))  # Shape for LSTM
    return X, np.array(y)

# Build the LSTM model
def build_model():
    """
    Build the LSTM model with a specified architecture.
    Returns the compiled model.
    """
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Forecasting function
def forecast_next_minutes(model, last_price, n_minutes=5):
    """
    Forecast the next n minutes of XRP prices using the trained model.
    Returns a list of predicted prices.
    """
    predictions = []
    current_data = np.array(last_price).reshape((1, 1, 1))
    for _ in range(n_minutes):
        next_prediction = model.predict(current_data)
        predictions.append(next_prediction[0, 0])
        current_data = next_prediction.reshape((1, 1, 1))
    return predictions

# Function to append forecast with processed time and save to CSV
def save_forecast_with_timestamp(forecast_df):
    """
    Append the processed_time column with the current timestamp and save to CSV.
    """
    forecast_df['processed_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Append mode if file exists, otherwise create a new file
    if os.path.exists(forecast_file):
        forecast_df.to_csv(forecast_file, mode='a', header=False, index=False)
    else:
        forecast_df.to_csv(forecast_file, index=False)
    print(f"Forecast saved with processed time to {forecast_file}.")

# Main loop for continuous forecasting
def run_forecasting_loop():
    """
    Main loop to continually forecast XRP prices every 5 minutes.
    Loads historical data, trains the model, and makes predictions using live data.
    """
    # Load historical data
    historical_data = load_historical_data()
    X_train, y_train = preprocess_for_training(historical_data)

    # Build the LSTM model
    model = build_model()

    # Check if weights exist and load them
    if os.path.exists(model_weights_file):
        print("Loading existing model weights...")
        model.load_weights(model_weights_file)

    # Callbacks for checkpointing and early stopping
    checkpoint = ModelCheckpoint(model_weights_file, save_best_only=True, save_weights_only=True)
    early_stopping = EarlyStopping(monitor='loss', patience=5)  # Stop if no improvement for 5 epochs

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, callbacks=[checkpoint, early_stopping])
    model.save(model_file)

    while True:
        # Load current day data
        current_day_data = pd.read_csv(current_day_file, parse_dates=['Timestamp'])
        
        # Check if data is available
        if not current_day_data.empty:
            latest_price = current_day_data['XRP Price'].iloc[-1]
            next_5_minutes_predictions = forecast_next_minutes(model, latest_price)

            # Save forecasted values with processed_time
            forecast_df = pd.DataFrame(next_5_minutes_predictions, columns=['Forecasted XRP Price'])
            forecast_df['Timestamp'] = pd.date_range(start=pd.Timestamp.now(), periods=5, freq='T')
            save_forecast_with_timestamp(forecast_df)

            # Display the last run time
            last_run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"Forecasted values for the next 5 minutes saved to: {forecast_file}")
            print(f"Last run time: {last_run_time}")
        else:
            print("Current day data is empty. Please check the input file.")
        
        # Further training on current day data
        X_live, y_live = preprocess_for_training(current_day_data)
        model.fit(X_live, y_live, epochs=1, batch_size=32, callbacks=[checkpoint, early_stopping])

        # Wait for 5 minutes before the next forecast
        time.sleep(300)  # Sleep for 5 minutes

# Run the forecasting loop
if __name__ == "__main__":
    run_forecasting_loop()

Loaded historical data successfully.
Loading existing model weights...
Epoch 1/5
29685/29685 ━━━━━━━━━━━━━━━━━━━━ 68s 2ms/step - loss: 7.1828e-07
Epoch 2/5
29685/29685 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 4.3861e-07
Epoch 3/5
29685/29685 ━━━━━━━━━━━━━━━━━━━━ 83s 2ms/step - loss: 4.4884e-07
Epoch 4/5
29685/29685 ━━━━━━━━━━━━━━━━━━━━ 60s 2ms/step - loss: 4.4502e-07
Epoch 5/5
29685/29685 ━━━━━━━━━━━━━━━━━━━━ 66s 2ms/step - loss: 4.4858e-07


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Forecast saved with processed time to E:\Data_Practice\xrp_forecast.csv.
Forecasted values for the next 5 minutes saved to: E:\Data_Practice\xrp_forecast.csv
Last run time: 2024-10-24 21:28:48
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4865e-07
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Forecast saved with processed time to E:\Data_Practice\xrp_forecast.csv.
Forecasted values for the next 5 minutes saved to: E:\Data_Practice\xrp_forecast.csv
Last run time: 2024-10-24 21:33:49
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1077e-07
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step